## case 2. Fully Character-Level Machine Translation

- Neural Net 입장에서는 문장이란 어떤 것인가?
    - 우리가 보기에 문장이란 단어로 이루어졌다고 생각할 수 있지만, 실제로 Neural Net은 문장을 결국에는 Continuous Vector Space의 형태로 읽는다.
    - 문장을 단어로 보든, character의 seq로 보든, 형태소로 보든, 문장에 이모지를 합쳐서 생각하든, 그림으로 보든, Encoding만 하면 된다.
    - 그러면, Preprocessing을 최소로 할 수 있는 것이 어디까지인가.
    

### Sentence를 연구한 단계들

- a) A sequence of words?: (Welcome, to, Montreal, !)
    - cons: vocab이 커짐, 특히 compound word(합성어)를 고려할 때
- b) A sequence of subwords?:  (Wel, come, to, Mont, real, !)
    - segmentation을 이용해 subword를 써보자. phrase-based translation을 할 때는 segmentation을 어떻게하는 지가 굉장히 성능에 영향을 많이 미쳤었는데, NN을 쓰면서
    - segmentation의 성능은 중요하지 않아 가장 간단한 BPE(Byte paire encoding)을 사용해도 된다.
- c) A sequence of sequences of letters?: ((W,e,l,c,o,m,e),(t,o),(M,o,n,t,r,e,al),(!))
    - 문장을 sequences of letters로 보아 Hierarchical RNN(RNN위에 RNN을 돌리는)을 하는 식으로 하는 것이다. 근데 코딩할 때 너무 불편해 교수님은 선호하지 않음.
- d) A sequence of characters?: (W,e,l,c,o,m,e, ,t,o, ,M,o,n,t,r,e,a,l,!)
    - 그냥 letter sequnce로 보는 것.
    - 2015, 16년도에는 c)와 d)를 중점적으로 연구했었다.
- e) A sequence of bits…?



### Why not (sub)word-level modelling?

- d)를 연구하기 위해, 왜 c)와 d)는 안좋은지, 그리고 왜 d)들로 가야하는지 생각해 보았다. 시간과 자원을 낭비하지 않기 위해.


- (sub)word-level modelling로 할 때의 단점들:
- 1) Difficult to handle morphology(형태)
    - 형태가 다양한 언어는 합성어를 너무 쉽게 만들어, vocab이 너무 많이 늘어난다.
        - ex) 과학/기술/정보/통신/부: (Rare morphological variants)
- 2) Difficult to handle misspelling
    - sns에서의 단어들, 줄임말들의 패턴을 알아내기 힘들다.
        - ex) could => cld
- 3) Modelling inefficiency
    - "kolmi/vaihe/kilo/watti/tunti/mittari": one vector?
        - 3000 킬로 와트 시간당 미터와같은 말에서
        - 맨 앞의 "kolme": one vector???로 하는 것이
        - 이 단어 하나하나마다 똑같이 파라미터를 주는게 맞는가.
        


### Problems with character-level modelling

- 그래서 교수님이 생각하고 다른 사람들이 문제를 제기한 것들. 그러면 character-level modelling은 무슨 문제가 있는가. (아직도 이런 문제가 존재한다.) 
- 가장 3가지 Major한 문제들
    - 1) Can a neural network generate a long, coherent(일관성 있는) sequence?
        - NN을 사용할 때 길면서도 말이 되는 seq를 만들 수 있는가.
    - 2) Can a neural network capture highly nonlinear orthography?
        - 단어는 이미 high-level abstraction이 잘 되어있다. 
        - ex) quit, quite, queit은 spelling 상으로는 유사하지만 완전 다른 문제
    - 3) Can character-level modelling be done efficiently?
        - character-level로 가면 length가 쭉 늘어남. 이를 어떻게 효율적으로 프로세스할 것인가.
    

- 그리고 위 문제들을 하나씩 해결해보자.


- 1) Generating a long, coherent sequence
    - source 부분에서는 당시 standard였던 subword (BPE) sequence 
    - Target 부분을 character sequence하게 나오게.
    - 결과: <img  src="./image/img_c2-1.PNG" width="50%"> 
    - 해석: Attention과 Recurrent를 합치면 아무리 길어도 (char로 바꾸면 len이 200정도로 길어지니까) coherent하게 generate 가능함을 알았다.
    - 이제 source side도 바꾸어 보자.

- 2) Going fully character-level 
    - source side 까지 바꿀 때 문제점:
        - quit, quite, queit과 같은 Orthography(철자법)의 aritrary한 pattern은 어떻게 capture할 것인가.
        - Complexity of attention grows quadratically: 시간복잡도
        (BPE-to-BPE: 30 x 30, BPE-to-Char: 30 x 120, Char-to-Char: 120 x 120) 
    - 위 2 문제를 동시에 해결하는 방법으로 Sentence Representation을 할 때
        - Convolution과 RNN을 같이 사용하는 아이디어가 있다.

<img  src="./image/img_c2-2.PNG" width="80%">

- a) Multi-width convolution for a character sequence [Kim et al., 2015]    
    - local한 정보 캡처를 잘하는 Convolution을 활용한다. 이때 width를 다양하게줘서.
    - 그리고 overlap된게 많아 나온 벡터들을 다 쓸 필요 없으니, Max pooling으로 sub-sampling. 
    - 이러면 모델링에서 local pattern을 잘 캡처하고, pooling으로 Efficiency도 올라가는 효과가 있다.
        - 길이가 길어질 때 이러한 방법으로 해결하는 것이다. 잘 섞어서.
- b) High-way network for nonlinear mapping [Srivastava et al., 2015]
    - 비선형을 위한 deep한 NN 사용
- c) Attention over a convolutionalfeature map [Xu et al., 2015]
    - Bidirectional RNN(?) 그림 상
    - 이러면 이전 층들에서 많이 줄어 들었어서 BPE를 활용한 것 보다(?) 효율이 좋을 때도 있다. 
<br>    
    
- 결과: 
    - 결과가 좋았다. Adequacy는 human evaluation.
<img  src="./image/img_c2-3.PNG" width="70%">
<br>
<img  src="./image/img_c2-4.PNG" width="80%">    

- (a) spelling mistake 에서도 robust하고
- (b)와 같은 처음보는 compound word에서도 자연스럽게 잘라 잘 번역했다.
- (c) trainset에 없었던 morphological variant도 잘 번역했다.
- (d) "verschlimmbesserung"라는 독어는 "worsend improvement"로 더 나빠지는 향상이라는 의미인 trainset에 없는 것인데, 이를 기계도 "worsening improvement"로 잘 번역했다.


- 그리고 Senrich 교수님이 좀 더 fine grained된 eval을 해본 것.
    - 어떤 것에서는 subword level이 잘하고, 어떤 것에서는  char level이 잘하는 즉 문제에 따라 다르다는 결론
<img  src="./image/img_c2-5.PNG" width="90%">      

- 이 결과를 보고, input을 여러 개를 동시에 쓰거나 서로 다른 종류를 한 번에 동시에 줄 수있나. 하는 질문이 생김.
    - 아까 말한 것 처럼, decoder는 source language가 아닌 vector representation만 보니까.
- $ \to $ ** Can we map multiple languages on a single continuous space? **


### Multilingual Neural Machine Translation
    
<img  src="./image/img_c2-6.PNG" width="70%">  

- 사람의 3개국어가 4개국어자와 같이 모델링 해보는 것이다,
- Source Language에 상관 없이 문장을 받아, Continuous vector space에 project하고 어떤 언어가 와도 번역할 수 있는 시스템.

- char level이나 점점 raw level로 갈수록 language사이에 어떤 방식으로 쓰는 지가 겹치는게 늘어난다.
    - ex) 알파벳에서는 indo european languages들(slavic, latin, gaelic 알파벳) 이지만 대부분 1 to 1매핑이 된다.

- 결과: {De, Cs, Fi, Ru} => En 그리고  No language ID
    - 특히나 데이터가 적은 Fi에서 잘되는 것을 볼 수 있고, (Prevents overfitting with low-resource language pairs)
    - Neural Network이 서로다른 modality(양식)의 input에서 그 사이의 shared되는 구조를 바로 continuous vector space에서 쓸 수 있는 것을 의미한다.
<img  src="./image/img_c2-7.PNG" width="50%"> 

- 문장에 섞어서 넣어 나온 결과
    - subword도 되긴한다.
    - 구글도 실제로 요즘 이렇게 어느 정도 합쳐서 설계해 실제 production에서 사용한다.
    - 진짜 마지막에는 4000개의 언어를 자유자재로 할 수 있지 않을까 하는게 장점이다.
<img  src="./image/img_c2-8.PNG" width="90%"> 




### Reference: 

- https://www.edwith.org/deepnlp 조경현교수님, 딥러닝을 이용한 자연어 처리 강의 및 강의 자료 


### Papers:

- Jason Lee, et al. "Fully Character-Level Neural Machine Translation without Explicit Segmentation", 2016
    - https://arxiv.org/abs/1610.03017
    
- Thanh-Le Ha, et al. "Toward Multilingual Neural Machine Translation with Universal Encoder and Decoder", 2016
    - https://arxiv.org/abs/1611.04798
    
- Melvin Johnson, et al. "Google's Multilingual Neural Machine Translation System: Enabling Zero-Shot Translation", 2017
    - https://transacl.org/ojs/index.php/tacl/article/view/1081

